In [1]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.dummy import DummyRegressor
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pickle
import catboost

In [3]:
# Change work directory
work_dir = r'C:\Users\tangc\OneDrive\Desktop\Files\HW\Datathon'
print(os.listdir(work_dir))

['.ipynb_checkpoints', '1', '2', 'EDA.ipynb', 'Finite_Element_Model.png', 'Introduction.ipynb', 'Joined_Melt_Pool', 'Meltpool.png', 'meltpool_1_nn.ipynb', 'meltpool_2_nn.ipynb', 'Merging data.ipynb', 'MergingData_TempPrediction.ipynb', 'my_logs', 'prediction_submission.csv', 'Predict_1b.ipynb', 'README.docx', 'saved_models', 'temperature_2_nn.ipynb']


# Task 1

## 1a

In [8]:
model_1a = pickle.load(open(r"saved_models\cat_1a.sav", 'rb'))

In [29]:
print(os.listdir(work_dir + r"\1\Testing")[0])
float(os.listdir(work_dir + r"\1\Testing")[0][0] + "." + os.listdir(work_dir + r"\1\Testing")[0][2:4])

0_62ms_110W.csv


0.62

In [55]:
# Predict 1a
for file in os.listdir(work_dir + r"\1\Testing")[:-1]:
    df_temp = pd.read_csv(work_dir + f"\\1\\Testing\\{file}").drop(["Unnamed: 4"], axis=1)
    df_temp["power"] = float(file[-8:-5])
    df_temp["speed"] = float(file[0] + "." + file[2:4])
    y_pred_temp = model_1a.predict(df_temp)
    df_temp["Temp"] = y_pred_temp
    df_temp.drop(["power", "speed"], axis=1, inplace=True)
    df_temp.to_csv(work_dir + f"\\1\\Testing\\{file}", index=False)

## 1b

In [56]:
# fuse csv for 1b
df_full = pd.DataFrame()
for file in os.listdir(work_dir + r"\1\Testing")[:-1]:
    df_temp = pd.read_csv(work_dir + f"\\1\\Testing\\{file}")
    df_temp["power"] = float(file[-8:-5])
    df_temp["speed"] = float(file[0] + "." + file[2:4])
    df_full = pd.concat([df_full, df_temp])
print(df_full.shape)
df_full.head()

(91200, 6)


,X_Coord,Y_Coord,Z_Coord,Temp,power,speed
0,0.18,-0.21,0.0,0.000292,110.0,0.62
1,0.15,-0.21,0.0,0.000310,110.0,0.62
2,0.12,-0.21,0.0,0.000269,110.0,0.62
3,0.09,-0.21,0.0,0.000249,110.0,0.62
4,0.06,-0.21,0.0,0.000159,110.0,0.62


In [68]:
model_1b_depth = pickle.load(open(r"saved_models\cat_1b_depth.sav", 'rb'))
model_1b_width = pickle.load(open(r"saved_models\cat_1b_width.sav", 'rb'))
model_1b_length = pickle.load(open(r"saved_models\cat_1b_length.sav", 'rb'))
X_test = pd.read_csv(work_dir + r"/1/Testing/Melt_Pool_Dimensions.csv")

In [69]:
# merge to 1b test
X_test_full = pd.merge(df_full, X_test, how='left', left_on=['speed','power'], 
                  right_on = ['Laser_Speed','Laser_Power'])
X_test_to_pred = X_test_full.drop(["Melt_Pool_Length", "Melt_Pool_Width", "Melt_Pool_Depth"], axis=1)

In [70]:
y_test_pred_depth = model_1b_depth.predict(X_test_to_pred)
y_test_pred_width = model_1b_width.predict(X_test_to_pred)
y_test_pred_length = model_1b_length.predict(X_test_to_pred)

In [77]:
X_test_full["Melt_Pool_Length"] = y_test_pred_length
X_test_full["Melt_Pool_Width"] = y_test_pred_width
X_test_full["Melt_Pool_Depth"] = y_test_pred_depth
df_1b_write = pd.DataFrame(X_test_full.groupby(["Laser_Speed", "Laser_Power"])["Melt_Pool_Length", "Melt_Pool_Width", "Melt_Pool_Depth"].mean()).reset_index()
df_1b_write.to_csv(work_dir + "\\1\\Testing\\Melt_Pool_Dimensions.csv", index=False)

# Task 2

## 2a

In [80]:
model_2a = pickle.load(open(r"saved_models\cat_900_0.07_14.sav", 'rb'))

In [99]:
print(os.listdir(work_dir + r"\2\Testing")[:-1])

['0_62ms_110W_20Deg.csv', '0_62ms_110W_40Deg.csv', '0_62ms_110W_60Deg.csv', '0_62ms_110W_80Deg.csv', '0_62ms_160W_20Deg.csv', '0_62ms_160W_40Deg.csv', '0_62ms_160W_60Deg.csv', '0_62ms_160W_80Deg.csv', '0_62ms_240W_20Deg.csv', '0_62ms_240W_40Deg.csv', '0_62ms_240W_60Deg.csv', '0_62ms_240W_80Deg.csv', '0_62ms_310W_20Deg.csv', '0_62ms_310W_40Deg.csv', '0_62ms_310W_60Deg.csv', '0_62ms_310W_80Deg.csv', '0_62ms_390W_20Deg.csv', '0_62ms_390W_40Deg.csv', '0_62ms_390W_60Deg.csv', '0_62ms_390W_80Deg.csv', '0_77ms_110W_20Deg.csv', '0_77ms_110W_40Deg.csv', '0_77ms_110W_60Deg.csv', '0_77ms_110W_80Deg.csv', '0_77ms_160W_20Deg.csv', '0_77ms_160W_40Deg.csv', '0_77ms_160W_60Deg.csv', '0_77ms_160W_80Deg.csv', '0_77ms_240W_20Deg.csv', '0_77ms_240W_40Deg.csv', '0_77ms_240W_60Deg.csv', '0_77ms_240W_80Deg.csv', '0_77ms_310W_20Deg.csv', '0_77ms_310W_40Deg.csv', '0_77ms_310W_60Deg.csv', '0_77ms_310W_80Deg.csv', '0_77ms_390W_20Deg.csv', '0_77ms_390W_40Deg.csv', '0_77ms_390W_60Deg.csv', '0_77ms_390W_80Deg.csv',

In [93]:
print(os.listdir(work_dir + r"\2\Testing")[0])
file = os.listdir(work_dir + r"\2\Testing")[0]
file[7:10]

0_62ms_110W_20Deg.csv


'110'

In [119]:
for file in os.listdir(work_dir + r"\2\Testing")[:-1]:
    df_temp = pd.read_csv(work_dir + f"\\2\\Testing\\{file}").drop(['Temp'], axis=1)
    df_temp["LasAngle"] = float(file[-9:-7])
    df_temp["LasPower"] = float(file[7:10])
    df_temp["LasSpeed"] = float(file[0] + "." + file[2:4])
    y_pred_temp = model_2a.predict(df_temp)
    df_temp["Temp"] = y_pred_temp
    df_temp.drop(["LasPower", "LasSpeed", "LasAngle"], axis=1, inplace=True)
    df_temp.to_csv(work_dir + f"\\2\\Testing\\{file}", index=False)

In [101]:
# Predict 2a
for file in os.listdir(work_dir + r"\2\Testing")[:-1]:
    df_temp = pd.read_csv(work_dir + f"\\2\\Testing\\{file}").drop(['Temp'], axis=1)
    df_temp["LasAngle"] = float(file[-9:-7])
    df_temp["LasPower"] = float(file[7:10])
    df_temp["LasSpeed"] = float(file[0] + "." + file[2:4])
    y_pred_temp = model_2a.predict(df_temp)
    df_temp["Temp"] = y_pred_temp
    df_temp.drop(["LasPower", "LasSpeed", "LasAngle"], axis=1, inplace=True)
    df_temp.to_csv(work_dir + f"\\2\\Testing\\{file}", index=False)

## 2b

In [112]:
model_2b_depth = pickle.load(open(r"saved_models\cat_2b_depth.sav", 'rb'))
model_2b_width = pickle.load(open(r"saved_models\cat_2b_width.sav", 'rb'))
model_2b_length = pickle.load(open(r"saved_models\cat_2b_length.sav", 'rb'))
X_2_test = pd.read_csv(work_dir + r"/2/Testing/Melt_Pool_Dimensions.csv")
X_2_test.head()

,Laser_Speed,Laser_Power,Edge_Angle,Edge_Distance,Laser_Direction,Melt_Pool_Length,Melt_Pool_Width,Melt_Pool_Depth
0,0.62,110,80,0.0758,-1,0.0,0.0,0.0
1,0.62,110,80,0.1064,-1,0.0,0.0,0.0
2,0.62,110,80,0.1369,-1,0.0,0.0,0.0
3,0.62,110,80,0.1674,-1,0.0,0.0,0.0
4,0.62,110,80,0.1979,-1,0.0,0.0,0.0


In [110]:
y_2_test_pred_depth = model_2b_depth.predict(X_2_test)
y_2_test_pred_width = model_2b_width.predict(X_2_test)
y_2_test_pred_length = model_2b_length.predict(X_2_test)

In [ ]:
    X_2_test["Melt_Pool_Length"] = y_2_test_pred_length
    X_2_test["Melt_Pool_Width"] = y_2_test_pred_width
    X_2_test["Melt_Pool_Depth"] = y_2_test_pred_depth
    X_2_test.to_csv(work_dir + "\\2\\Testing\\Melt_Pool_Dimensions.csv", index=False)